# Pointwise
* predict the relevance scores

# Import

In [15]:
from collections import Counter
from Evaluations import evaluation_metrics
import pandas as pd
import numpy as np

# models
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMRanker


In [16]:
RANDOM_STATE = 24

In [17]:
# 導入資料
train_df = pd.read_pickle('../data/Train_by_postoal_code_without_review_pointwise_v3_4.pkl').reset_index(drop=True)
test_df = pd.read_pickle('../data/Test_by_postoal_code_without_review_pointwise_v3_4.pkl').reset_index(drop=True)

In [18]:
##### Normalization 
#### 要記得做！！！
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

In [19]:
# expr. 1 change relevance score to binary
def change_rel_score(df):
    df['binary_score'] = ''
    for idx, row in df.iterrows():
        if row.relevance > 0.0:
            df['binary_score'][idx] = 1
        else:
            df['binary_score'][idx] = 0
    return df
train_df = change_rel_score(train_df)
test_df = change_rel_score(test_df)

/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
train_df['binary_score'] = train_df['binary_score'].astype('float')
test_df['binary_score'] = test_df['binary_score'].astype('float')

In [21]:
label_list = ['binary_score' ,'affinity', 'complementary','cosine_sim']
for i in label_list : 
    train_df[i] = train_df[i].astype('float')
    test_df[i] = test_df[i].astype('float')

KeyError: 'affinity'

In [22]:
train_df = train_df.sample(frac=1 , random_state = RANDOM_STATE)
test_df = test_df.sample(frac=1 , random_state = RANDOM_STATE)

In [23]:
train_df.shape

(22720, 11)

In [24]:
test_df.shape

(14240, 11)

# Models

In [25]:
# create list of list for query ranking
def get_ranking(df , y_label = 'binary_score'):
    
    """
    Turn the probability array into a list of lists for calculation.
    
    Parameters:
    df(DataFrame): the test dataframe
    
    Returns:
    prediction (list of lists): A list of predicted rankings for each query.
    actual (list of lists): A list of actual rankings for each query.
    """
 
    pred_list = []
    true_rel = []
    true_list = []

    output_dict = {}
    
    
    for id in Counter(df.business_id):

        output_dict[id] = {}
        
        tmp = df[df.business_id == id]
        a_sorted = tmp.sort_values(by=[y_label],ascending=[False])
        p_sorted = tmp.sort_values(by=['predictions'],ascending=[False])
        # p_sorted = p_sorted[p_sorted.predictions>0]

        true_list.append(list(a_sorted[a_sorted.relevance!=0].postal_code))
        pred_list.append(list(p_sorted.postal_code))
        true_rel.append(list(a_sorted[a_sorted.relevance!=0].relevance))
        
        output_dict[id]['predict'] = list(p_sorted.postal_code)
        output_dict[id]['true'] = list(a_sorted[a_sorted.relevance!=0].postal_code)
        
        
    return pred_list, true_rel , true_list , output_dict

# Model

In [26]:
# models
LR = LogisticRegression(random_state=RANDOM_STATE)
RF = RandomForestClassifier(random_state=RANDOM_STATE)
DTC = DecisionTreeClassifier(random_state=RANDOM_STATE)
KNC = KNeighborsClassifier()
SVC = svm.SVC(random_state=RANDOM_STATE)
GNB = GaussianNB()
LGBM = LGBMRanker(objective="lambdarank",random_state=RANDOM_STATE)

train_features=['density', 'entropy', 'competitiveness','area_pop']
# train_features=['density', 'entropy', 'competitiveness','area_pop','complementary','affinity']
# train_features=['density', 'entropy', 'competitiveness','area_pop', 'accessibility','cosine_sim']
# train_features=['density', 'entropy', 'competitiveness','area_pop','affinity', 'complementary','cosine_sim']


get_group_size = lambda df: df.reset_index().groupby("business_id")['business_id'].count()

train_groups = get_group_size(train_df).to_numpy()
test_groups = get_group_size(test_df).to_numpy()

print(sum(train_groups) , sum(test_groups))

22720 14240


In [27]:
models = [LR, RF, DTC, KNC, SVC, GNB, LGBM]
model_name =['LR', 'RF', 'DTC', 'KNC', 'SVC', 'GNB' , 'LGBMRanker'] 
score_dict = {}

for i in range(len(models)):
    score_dict[model_name[i]]={}
    model = models[i]
    # Train
    if model_name[i] != 'LGBMRanker':
        model.fit(scaler.fit_transform(train_df[train_features]), train_df[['binary_score']])
    else:
        model.fit(scaler.fit_transform(train_df[train_features]), train_df[['relevance']], group=train_groups)
    
    # Predict
    predict = model.predict(scaler.fit_transform(test_df[train_features]))
    test_df['predictions'] = predict
    if model_name[i] != 'LGBMRanker':
        pred_list, true_rel , true_list , output_dict = get_ranking(test_df)
    else:
        pred_list, true_rel , true_list , output_dict = get_ranking(test_df,'relevance')
        

    # Evaluation
    mrr_list = evaluation_metrics._mrr(pred_list , true_list)
    map_list = evaluation_metrics._map(pred_list , true_list)
    dcg_list , ndcg_list = evaluation_metrics._dcg_ndcg(pred_list , true_list,true_rel)
    precision_list_1 = evaluation_metrics._precision(pred_list , true_list, k=1)
    recall_list_1 = evaluation_metrics._recall(pred_list , true_list,k=1)
    precision_list_3 = evaluation_metrics._precision(pred_list , true_list, k=3)
    recall_list_3 = evaluation_metrics._recall(pred_list , true_list,k=3)
    precision_list =evaluation_metrics. _precision(pred_list , true_list)
    recall_list = evaluation_metrics._recall(pred_list , true_list)

    score_dict[model_name[i]]['precision @ 1'] = np.mean(precision_list_1)
#     score_dict[model_name[i]]['recall @ 1'] = np.mean(recall_list_1)
    score_dict[model_name[i]]['precision @ 3 '] = np.mean(precision_list_3)
    score_dict[model_name[i]]['recall @ 3'] = np.mean(recall_list_3)
    score_dict[model_name[i]]['precision'] = np.mean(precision_list)
    score_dict[model_name[i]]['recall'] = np.mean(recall_list)
    score_dict[model_name[i]]['mrr'] = np.mean(mrr_list)
    score_dict[model_name[i]]['map'] = np.mean(map_list)
    score_dict[model_name[i]]['dcg'] = np.mean(dcg_list)
    score_dict[model_name[i]]['ndcg'] = np.mean(ndcg_list)

/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y wa

In [28]:
pd.DataFrame(score_dict).round(3)

,LR,RF,DTC,KNC,SVC,GNB,LGBMRanker
precision @ 1,0.051,0.164,0.124,0.347,0.111,0.208,0.368
precision @ 3,0.051,0.151,0.114,0.152,0.069,0.121,0.207
recall @ 3,0.153,0.454,0.341,0.456,0.206,0.364,0.622
precision,0.050,0.050,0.050,0.050,0.050,0.050,0.050
recall,1.000,1.000,1.000,1.000,1.000,1.000,1.000
mrr,0.181,0.351,0.298,0.451,0.233,0.343,0.537
map,0.181,0.351,0.298,0.451,0.233,0.343,0.537
dcg,3.415,4.782,4.369,5.542,3.832,4.736,6.329
ndcg,0.353,0.494,0.452,0.567,0.394,0.483,0.645


In [ ]:
# pd.DataFrame(output_dict).T.to_csv('./output.csv')

In [ ]:
pd.DataFrame(output_dict).T

In [ ]:
one_df = train_df[train_df.binary_score==1]
zero_df = train_df[train_df.binary_score==0]

In [ ]:
one_df.describe()

In [ ]:
zero_df.describe()

In [ ]:
train_df[train_df.business_id=='SyiPLENX6wMfagzrsw2aFg']

In [ ]:
train_df[train_df.business_id == 'EvdYKS6NVnU6J7emNJV2BQ']